In [268]:
import pandas as pd
import numpy as np

df = pd.read_csv('../titanic_data/train.csv')
test = pd.read_csv('../titanic_data/test.csv')
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [275]:
df['Sex'] = df['Sex'].astype('category').cat.codes
df['Embarked'] = df['Embarked'].astype('category').cat.codes
df['Cabin'] = df['Cabin'].astype('category').cat.codes
df['Name_length'] = df.Name.apply(len)

df['ms'] = df['Name'].str.contains("Miss. ")
df['mr'] = df['Name'].str.contains("Mr. ")
df['mrs'] = df['Name'].str.contains("Mrs. ")
df['master'] = df['Name'].str.contains("Master. ")
df['parenth'] = df['Name'].str.contains("\(")
df['nickname'] = df['Name'].str.contains('\"')

def get_element(my_list, position):
    return my_list[position]

df['ticket_num'] = df.Ticket.str.split(' ').apply(get_element, position=-1)
df['ticket_num'] = df['ticket_num'].apply(pd.to_numeric,errors='coerce')

df['ticket_num'].fillna(0, inplace=True)
df['Age'].fillna(150, inplace=True)


In [282]:
from sklearn.model_selection import train_test_split

X = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Name_length', 'ticket_num', 'Cabin',
       'ms', 'mrs', 'mr', 'master', 'parenth', 'nickname']]
#X = df[['Pclass', 'Sex', 'Age', 'Fare', 'Name_length', 'ticket_num']]
X = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Fare', 'Name_length', 'ticket_num', 'Cabin',
       'ms', 'mrs', 'parenth']]
y = df['Survived']

#print(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 141)

In [290]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

clf = RandomForestClassifier(n_estimators=100, max_depth = 5, random_state=47).fit(X_train, y_train)

y_predicted = clf.predict(X_test)

print('Recall: {:.3f}'.format(recall_score(y_test, y_predicted)))
print('Precision: {:.3f}'.format(precision_score(y_test, y_predicted)))
print('Accuracy: {:.3f}'.format(accuracy_score(y_test, y_predicted)))
print('F1: {:.3f}'.format(f1_score(y_test, y_predicted)))
confusion = confusion_matrix(y_test, y_predicted)
print(confusion)
print('Feature importances: {}'.format(clf.feature_importances_))

ValueError: Found input variables with inconsistent numbers of samples: [223, 418]

In [272]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(n_estimators=100, learning_rate = 0.1, 
                                 max_depth = 5, random_state=37).fit(X_train, y_train)

y_predicted = clf.predict(X_test)
confusion = confusion_matrix(y_test, y_predicted)

print('Recall: {:.3f}'.format(recall_score(y_test, y_predicted)))
print('Precision: {:.3f}'.format(precision_score(y_test, y_predicted)))
print('Accuracy: {:.3f}'.format(accuracy_score(y_test, y_predicted)))
print('F1: {:.3f}'.format(f1_score(y_test, y_predicted)))
confusion = confusion_matrix(y_test, y_predicted)
print(confusion)
print('Feature importances: {}'.format(clf.feature_importances_))



Recall: 0.782
Precision: 0.819
Accuracy: 0.848
F1: 0.800
[[121  15]
 [ 19  68]]
Feature importances: [0.029 0.026 0.127 0.016 0.013 0.152 0.014 0.145 0.309 0.054 0.004 0.013
 0.082 0.003 0.004 0.007]


In [152]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
# we must apply the scaling to the test set that we computed for the training set
X_test_scaled = scaler.transform(X_test)

In [155]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

clf = SVC(kernel='rbf')
grid_values = {'gamma': [0.001, 0.01, 0.05, 0.1, 1, 10, 100], 'C': [0.01, 0.1, 1, 10, 100]}
grid_values = {'gamma': [0.001, 0.01, 0.05, 0.1, 1, 10, 100], 'C': [10, 30, 100, 300, 1000]}

grid_clf = GridSearchCV(clf, param_grid = grid_values, scoring = 'accuracy')
grid_clf.fit(X_train_scaled, y_train)
grid_clf.cv_results_['mean_test_score'].reshape(5,7)

array([[0.790, 0.789, 0.822, 0.814, 0.808, 0.765, 0.656],
       [0.790, 0.801, 0.819, 0.813, 0.805, 0.762, 0.650],
       [0.789, 0.822, 0.819, 0.807, 0.795, 0.741, 0.627],
       [0.801, 0.829, 0.810, 0.808, 0.769, 0.732, 0.617],
       [0.814, 0.817, 0.813, 0.804, 0.746, 0.701, 0.605]])

In [156]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

lr = LogisticRegression()
grid_values = {'penalty': ['l1', 'l2'], 'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]}
grid_lr = GridSearchCV(lr, param_grid = grid_values, scoring = 'recall').fit(X_train_scaled, y_train)
print(grid_lr.cv_results_['mean_test_score'].reshape(9,2))

[[0.000 0.000]
 [0.000 0.024]
 [0.000 0.510]
 [0.694 0.698]
 [0.722 0.718]
 [0.710 0.718]
 [0.710 0.714]
 [0.710 0.710]
 [0.710 0.710]]


In [199]:
from sklearn.neural_network import MLPClassifier

nnclf = MLPClassifier(hidden_layer_sizes = [5,5,5], solver='adam', alpha=0.0001, activation='identity',
                     max_iter = 1000, random_state = 47).fit(X_train_scaled, y_train)

y_predicted = nnclf.predict(X_test_scaled)
confusion = confusion_matrix(y_test, y_predicted)

print('Recall: {:.3f}'.format(recall_score(y_test, y_predicted)))
print('Precision: {:.3f}'.format(precision_score(y_test, y_predicted)))
print('Accuracy: {:.3f}'.format(accuracy_score(y_test, y_predicted)))
print('F1: {:.3f}'.format(f1_score(y_test, y_predicted)))
confusion = confusion_matrix(y_test, y_predicted)
print(confusion)

Recall: 0.759
Precision: 0.815
Accuracy: 0.839
F1: 0.786
[[121  15]
 [ 21  66]]


In [226]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 15)
knn.fit(X_train_scaled, y_train)

y_predicted = knn.predict(X_test_scaled)

confusion = confusion_matrix(y_test, y_predicted)

print('Recall: {:.3f}'.format(recall_score(y_test, y_predicted)))
print('Precision: {:.3f}'.format(precision_score(y_test, y_predicted)))
print('Accuracy: {:.3f}'.format(accuracy_score(y_test, y_predicted)))
print('F1: {:.3f}'.format(f1_score(y_test, y_predicted)))
confusion = confusion_matrix(y_test, y_predicted)
print(confusion)

Recall: 0.747
Precision: 0.855
Accuracy: 0.852
F1: 0.798
[[125  11]
 [ 22  65]]


In [284]:
test['Sex'] = test['Sex'].astype('category').cat.codes
test['Embarked'] = test['Embarked'].astype('category').cat.codes
test['Cabin'] = test['Cabin'].astype('category').cat.codes
test['Name_length'] = test.Name.apply(len)

test['ms'] = test['Name'].str.contains("Miss. ")
test['mr'] = test['Name'].str.contains("Mr. ")
test['mrs'] = test['Name'].str.contains("Mrs. ")
test['master'] = test['Name'].str.contains("Master. ")
test['parenth'] = test['Name'].str.contains("\(")
test['nickname'] = test['Name'].str.contains('\"')

def get_element(my_list, position):
    return my_list[position]

test['ticket_num'] = test.Ticket.str.split(' ').apply(get_element, position=-1)
test['ticket_num'] = test['ticket_num'].apply(pd.to_numeric,errors='coerce')

test['ticket_num'].fillna(0, inplace=True)
test['Age'].fillna(150, inplace=True)

In [299]:
test['Fare'].fillna(0, inplace=True)
# X_test = test[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Name_length', 'ticket_num', 'Cabin',
#        'ms', 'mrs', 'mr', 'master', 'parenth', 'nickname']]
X_test = test[['Pclass', 'Sex', 'Age', 'SibSp', 'Fare', 'Name_length', 'ticket_num', 'Cabin',
       'ms', 'mrs', 'parenth']]

#print(X_test)

#clf = RandomForestClassifier(n_estimators=100, max_depth = 5, random_state=47).fit(X, y)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X)
# we must apply the scaling to the test set that we computed for the training set
X_test_scaled = scaler.transform(X_test)

# knn = KNeighborsClassifier(n_neighbors = 15)
# knn.fit(X_train_scaled, y)

nnclf = MLPClassifier(hidden_layer_sizes = [5,5,5], solver='adam', alpha=0.0001, activation='identity',
                     max_iter = 1000, random_state = 47).fit(X_train_scaled, y)

y_predicted = nnclf.predict(X_test_scaled)

#y_predicted = clf.predict(X_test)



In [300]:
answer = pd.DataFrame(data=y_predicted, index = test['PassengerId'], columns=['Survived'])

print(answer)

             Survived
PassengerId          
892                 0
893                 1
894                 0
895                 0
896                 1
897                 0
898                 1
899                 0
900                 1
901                 0
902                 0
903                 0
904                 1
905                 0
906                 1
907                 1
908                 0
909                 0
910                 1
911                 1
912                 0
913                 0
914                 1
915                 0
916                 1
917                 0
918                 1
919                 0
920                 0
921                 0
...               ...
1280                0
1281                0
1282                0
1283                1
1284                0
1285                0
1286                0
1287                1
1288                0
1289                1
1290                0
1291                0
1292      

In [301]:
answer.to_csv('/Users/jcbrooks/kaggle/titanic_analysis/predictions.csv')